In [2]:
import sqlite3
import json
import pandas as pd

In [3]:
def query2df(sqlite_path, query):
    print("\n", query)
    conn = sqlite3.connect(sqlite_path)
    cursor = conn.execute(query)
    return pd.DataFrame(data=cursor.fetchall(), columns=[c[0] for c in cursor.description])

In [4]:
conn = sqlite3.connect('data/sample.db') # sqlite3.connect(":memory:")
c = conn.cursor()

In [5]:
def flatten(obj):
    if isinstance(obj, str):
        return "'" + obj + "'"
    else:
        return "'" + repr(obj) + "'"

In [7]:
c.execute("DROP TABLE students")
c.execute("CREATE TABLE IF NOT EXISTS students (id object PRIMARY KEY, date text)")
c.execute("INSERT INTO students(date) VALUES ('2006-01-05')")

c.execute("CREATE INDEX idx_id ON students(id)")

display(query2df('data/sample.db', "PRAGMA index_list('students')"))
display(query2df('data/sample.db', "PRAGMA table_info('students')"))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(query2df('data/sample.db', "SELECT name as table_name FROM sqlite_master WHERE type ='table' AND name NOT LIKE 'sqlite_%';"))

dic = {'new' : [1, 2, 4], 'date' : '2006-02-22', 'raw' : 'afd, (),'}
for key in dic.keys():
    try:
        c.execute(f"ALTER TABLE students ADD COLUMN {key}")
    except:
        pass


c.execute(f"INSERT INTO students({', '.join(dic.keys())}) VALUES ({', '.join('?' * len(dic.keys()))})", tuple(flatten(e) for e in dic.values()))

conn.commit()


 PRAGMA index_list('students')


,seq,name,unique,origin,partial
0,0,sqlite_autoindex_students_1,1,pk,0



 PRAGMA table_info('students')


,cid,name,type,notnull,dflt_value,pk
0,0,id,object,0,None,1
1,1,date,text,0,None,0



 SELECT name as table_name FROM sqlite_master WHERE type ='table' AND name NOT LIKE 'sqlite_%';


,table_name
0,stocks
1,students


In [30]:
for row in c.execute('SELECT * FROM students ORDER BY date'):
    print(row)

(None, '2006-01-05')


In [2]:
def db2dicts(db_path):
    db = bsddb3.btopen(db_path, 'r')
    for id in db:
        try:
            dic = json.loads(db[id])
            assert isinstance(dic, dict)
        except:
            dic = {'raw': db[id].decode('utf-8')}

        yield({'_' + k: v for k, v in dic.items()})

def json2server(db_path, sqlite_path=None):
    conn = sqlite3.connect(sqlite_path)
    try:
        conn.execute(f"DROP TABLE {cate}")
    except:
        pass

    conn.execute(f"CREATE TABLE IF NOT EXISTS {cate} (_id object PRIMARY KEY)")

    for dic in db2dicts(db_path):
        columns = set(e[1] for e in conn.execute(f"PRAGMA table_info('{cate}')").fetchall())
        for key in dic.keys():
            if key not in columns:
                conn.execute(f"ALTER TABLE {cate} ADD COLUMN {key}")
                columns.add(key)
        try:
            conn.execute(f"INSERT INTO {cate}({', '.join(dic.keys())}) VALUES ({', '.join('?' * len(dic.keys()))})", tuple(repr(e) for e in dic.values()))
        except:
            logger.warn("invalid" + str(sys.exc_info()[0]))

    conn.commit()

def query2df(sqlite_path, query, mode="df", commit=False):
    conn = sqlite3.connect(sqlite_path)
    cursor = conn.execute(query)

    if query.lower().startswith("select") or query.lower().startswith("pragma"):
        if mode == "df":
            return pd.DataFrame(data=cursor.fetchall(), columns=[c[0] for c in cursor.description])
        else:
            return cursor
    elif commit:
        conn.commit()